# GCN Deep Graph Infomax on CORA

This demo demonstrates how to perform unsupervised training of a GCN model using the Deep Graph Infomax algorithm (https://arxiv.org/pdf/1809.10341.pdf) on the CORA dataset. 

As with all StellarGraph workflows: first we load the dataset, next we create our data generators, and then we train our model. We then take the embeddings created through unsupervised training and predict the node classes using logistic regression.

In [1]:
from stellargraph.mapper import CorruptedFullBatchNodeGenerator
from stellargraph import StellarGraph
from stellargraph.layer import GCN, fullbatch_infomax_embedding_model, fullbatch_infomax_node_model
from stellargraph import datasets
from stellargraph.utils import plot_history

import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from IPython.display import display, HTML

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import Model

In [2]:
dataset = datasets.Cora()
display(HTML(dataset.description))
G, node_subjects = dataset.load()

 ## Data Generators
 
 Now we create the data generators using `CorruptedFullBatchNodeGenerator`. `CorruptedFullBatchNodeGenerator` returns shuffled node features along with the regular node features and we train our model to discriminate between the two. 
 
 Note that:
 
 - We typically pass all nodes to `generator.flow` because this is an unsupervised task
 - We don't pass `targets` to `generator.flow` because these are binary labels (true nodes, false nodes) that are created by `CorruptedFullBatchNodeGenerator`

In [3]:
generator = CorruptedFullBatchNodeGenerator(G,)
gen = generator.flow(G.nodes(),)

Using GCN (local pooling) filters...


## Model Creation and Training

We create an train our `GCNInfoMax` model. Note that the loss used here must always be `tf.nn.sigmoid_cross_entropy_with_logits`.

In [4]:
gcn = GCN([512], activations=[tf.keras.layers.PReLU()], generator=generator)

x_in, x_out = fullbatch_infomax_node_model(gcn)

model = Model(inputs=x_in, outputs=x_out)
model.compile(loss=tf.nn.sigmoid_cross_entropy_with_logits, optimizer=Adam(lr=1e-3))

/Users/kieranricardo/PycharmProjects/current-stellargraph/stellargraph/stellargraph/core/experimental.py:88: ExperimentalWarning: fullbatch_infomax_node_model is experimental: lack of unit tests (see: https://github.com/stellargraph/stellargraph/issues/1003). It may be difficult to use and may have major changes at any time.
  warnings.warn(direct_msg, ExperimentalWarning)


In [ ]:
es = EarlyStopping(monitor="loss", min_delta=0, patience=20)
history = model.fit(gen, epochs=1000, verbose=0, callbacks=[es])
plot_history(history)

  ...
    to  
  ['...']


## Extracting Embeddings and Logistic Regression

We create an embedding model using `GCNInfoMax.embedding_model` to obtain the node embeddings. Then we use logistic regression to predict which class the node belongs to.

In [ ]:
x_emb_in, x_emb_out = fullbatch_infomax_embedding_model(model)
emb_model = Model(inputs=x_emb_in, outputs=x_emb_out)

In [ ]:
train_subjects, test_subjects = model_selection.train_test_split(
    node_subjects, train_size=0.1, test_size=None, stratify=node_subjects
)

target_encoding = preprocessing.LabelBinarizer()

train_targets = target_encoding.fit_transform(train_subjects)
test_targets = target_encoding.transform(test_subjects)

test_gen = generator.flow(test_subjects.index)
train_gen = generator.flow(train_subjects.index)

test_embeddings = emb_model.predict(test_gen)
train_embeddings = emb_model.predict(train_gen)

lr = LogisticRegression(multi_class="auto", solver="lbfgs")
lr.fit(train_embeddings, train_subjects)

y_pred = lr.predict(test_embeddings)
print("Test classification accuracy:", (y_pred == test_subjects).mean())

In [ ]:
train_targets.shape

## Visualisation with TSNE

Here we visualize the node embeddings with TSNE. As you can see below, the Deep Graph Infomax model produces well separated embeddings using unsupervised training.

In [ ]:
all_embeddings = emb_model.predict(generator.flow(G.nodes()))

y = np.argmax(target_encoding.transform(node_subjects), axis=1)
trans = TSNE(n_components=2)
emb_transformed = pd.DataFrame(trans.fit_transform(all_embeddings), index=list(G.nodes()))
emb_transformed["label"] = y

In [ ]:
alpha = 0.7

fig, ax = plt.subplots(figsize=(7, 7))
ax.scatter(
    emb_transformed[0],
    emb_transformed[1],
    c=emb_transformed["label"].astype("category"),
    cmap="jet",
    alpha=alpha,
)
ax.set(aspect="equal", xlabel="$X_1$", ylabel="$X_2$")
plt.title("TSNE visualization of GCN embeddings for cora dataset")
plt.show()